In [ ]:

from datetime import date 
import datetime as dt 
from dateutil.relativedelta import relativedelta

#from..config import *

# KEY1 config["spark" ]["hadoop"]["key1"] 
# KEY2= config["spark"]["hadoop"]["key2"]

Class Sparky:
  """
  class to initialize spark and methods.

  Parameters
  --------
  env_keys:
    expects jupyter os.environ dict
  """
  def init__(self)-> SparkSession:
    self.spark self.spark_session()

  def spark_session(self): 
    hadoop_endpoint= 'https://s3.sgp.com'
    presto_endpoint= 'jdbc:presto://sgp.com:8888/hive/information_schema'
    s3_path= 's3a://user/{tdy}'

    session= (SparkSession
            .builder
            .config("spark.hadoop.ipc.client.fallback-to-simple-auth-allowed", "true")
            .config("spark.network. timeout", "180") .config("spark.dynamicAllocation.executorIdleTimeout", "128")
            .config("spark.kryoserializer.buffer.max", "1024m")
            .config("spark.driver.extrajavaoptions","Dcom.amazonaws.sdk.disableCertChecking-true") 
            .config("spark.executor. extrajavaoptions", Dcom.amazonaws.sdk.disableCertChecking-true")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFilesystem")
            .config("spark.hadoop.fs.s3a.access.key", os.environ["hadoop1"])
            .config("spark.hadoop.fs.s3a.secret.key", os.environ["hadoop2"]).
            .config("spark.hadoop.fs.s3a.endpoint", hadoop_endpoint) .appName ("PrestoConnect")
            .config("spark.executor.cores", 1) .config("spark.dynamicAllocation.minExecutors", e)
            .config("spark.dynamicAllocation.maxExecutors", 12)
            .config("spark.executor.memory", "2g') .config("spark.yarn.executor.memoryOverhead", "2g')
            .config("spark.yarn.queue","root.ADA_ANALYTICS_RMG")
            .config("spark.driver.memory", "2g"). .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
            .config("spark.files","/data/custom-jars/presto/presto.jks,/data/custom-jars/presto/truststore.jks,/etc/krbs.conf") 
            .config("spark.jars", "/data/custom-jars/presto/presto-jdbc-312-e.7.jar") 
            .getorCreate())
      return session

  def read_table_presto(self, table_name): 
      jdbcDF = self.spark.read \
              .format("jdbc") \
              .option("driver", "io.prestosql.jdbc.PrestoDriver") \
              .option("url", presto_endpoint) \
              .option("SSL","true") \
              .option ("SSLTrustStore Path", "/data/custom-jars/presto/truststore.jks") \
              .option("SSLTrustStorePassword", "changeit") \
              .option("SSLKeyStorePath","/data/custom-jars/presto/presto.jks") \
              .option("SSLKeyStore Password", "changeit") \
              .option("dbtable", table_name) \
              .option("username", os.environ["username"]) \
              .option("password", os.environ["password"]) \
              .load() 
      return jdbcDF

  def push_to_s3(self, query, table): 
    tdy= date.today().strftime('%Y%m%d')
    tmp_path = s3_path
    query.repartition(1).write\
      .option("mapreduce.fileouputcommitter.algorithm.version", "2")\
      .option("header", "true")\ .format('com.databricks.spark.csv')\
      .save(fr"{tmp_path}/{table}", mode = "overwrite")

    Path = sc. _gateway.jvm.org.apache.hadoop.fs.Path 
    fs = Path(tmp_path).getFileSystem (self.spark._jsc.hadoopConfiguration()) 
    csv_part_file = fs.globStatus (Path (f'{tmp_path}/{table}' + "/part*"))[0].getPath() 
    fs.rename(csv_part_file, Path (f' {tmp_path}/{table}.csv')) 
    fs.delete (Path (f'{tmp_path}/{table}'), True)

